# Pathology NN Analysis 

In [9]:
import os,sys
import numpy as np
from PIL import Image
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from tqdm import tqdm_notebook as tqdm

In [10]:
# read data
data_num = 11839 # pos:319, neg:924
data_size = 12288
label = np.zeros((11839))
data = np.zeros((11839, 12288))
with open('data.csv') as lines:
    for n, line in enumerate(tqdm(lines)):
        line = line.strip().split()
        label[n] = line[0]
        data[n,:] = line[1:]

In [11]:
# save to image
for n in tqdm(range(data.shape[0])):
    im = np.reshape(data[n,:], (3,64,64))
    im = np.swapaxes(im,0,2)
    im = np.swapaxes(im,0,1)
    im = im*128 + 127
    result = Image.fromarray((im).astype(np.uint8))
    result_resized = result.resize((299,299), resample=Image.BILINEAR)
    result_resized.save(os.path.join('data_resized','image'+str(n)+'.png'),'png')
    result.save(os.path.join('data','image'+str(n)+'.png'),'png')

In [12]:
print data.shape
print label.shape

(11839, 12288)
(11839,)


In [17]:
fp = open('all_labels.txt', 'wb')
for n, l in enumerate(label):
    fp.write(str(n) + ',' + str(int(l)) + '\n')

In [18]:
ls = []
with open('all_labels.txt') as lines:
    for line in lines:
        id, l = line.strip().split(',')
        ls.append(int(l))

In [33]:
pos = np.where(label==1)[0]
neg = np.where(label==0)[0]

In [34]:
print(pos.shape, neg.shape)

((1430,), (10409,))


In [113]:
# sample 3300+990 - 1430 = 2860 negatives
neg_idx = np.random.permutation(neg.shape[0])
neg_s = neg[neg_idx[:2860]]
label_neg = np.zeros(2860)
pos_s = pos
label_pos = np.ones(1430)
data_sampled = np.hstack((pos_s, neg_s))
label_sampled =np.hstack((label_pos, label_neg))
print data_sampled.shape, label_sampled.shape

(4290,) (4290,)


In [114]:
# shuffle and split
shuffle_idx = np.random.permutation(data_sampled.shape[0])
data_sampled = data_sampled[shuffle_idx]
label_sampled = label_sampled[shuffle_idx]

In [115]:
tr_data = data_sampled[:3300].tolist()
tr_label = label_sampled[:3300].tolist()
tt_data = data_sampled[3300:].tolist()
tt_label = label_sampled[3300:].tolist()

In [116]:
tr_fp = open('splits/sp7/tr_lst', 'wb')
tt_fp = open('splits/sp7/tt_lst', 'wb')
count = 0
for (d, l) in zip(tr_data, tr_label):
    tr_fp.write(str(int(d)) + ',' + str(int(l)) + '\n')
    count += 1
print(count)
for (d, l) in zip(tt_data, tt_label):
    tt_fp.write(str(int(d)) + ',' + str(int(l)) + '\n')

3300


In [104]:
print(label[tt_data[0:10]])
print(tt_label[0:10])

[ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [83]:
print(len(tt_label), len(tr_label))

(990, 3300)


In [40]:
# sample 3300+990 - 1430 = 2860 negatives
neg_idx = np.random.permutation(neg.shape[0])
data_neg = data[neg_idx[:2860]]
label_neg = np.zeros(2860)
data_pos = data[pos]
label_pos = np.ones(1430)
data_sampled = np.vstack((data_pos, data_neg))
label_sampled =np.hstack((label_pos, label_neg))
print data_sampled.shape, label_sampled.shape

(4290, 12288) (4290,)


In [41]:
# shuffle and split
shuffle_idx = np.random.permutation(data_sampled.shape[0])
data_sampled = data_sampled[shuffle_idx]
label_sampled = label_sampled[shuffle_idx]

In [45]:
# save dataset
tr_data = data_sampled[:3300]
tr_label = label_sampled[:3300]
tt_data = data_sampled[3300:]
tt_label = label_sampled[3300:]
np.save('splits/tr_data.npy', tr_data)
np.save('splits/tr_label.npy', tr_label)
np.save('splits/tt_data.npy', tt_data)
np.save('splits/tt_label.npy', tt_label)
print tr_data.shape, tt_data.shape

(3300, 12288) (990, 12288)
